In [9]:
# !pip install langgraph

In [10]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# state
- state is a global message used by the other nodes to get the context of the work done previously by other nodes or tools

In [11]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def search_tool(query: str):
    """search the web for info using DuckDuckGo."""
    search_tool = DuckDuckGoSearchRun()
    result = search_tool.invoke(query)
    return result


#add the tool into a node
tools = [search_tool]
tool_node = ToolNode(tools)

# setup llm node
model = ChatOllama(model="llama3.2:latest").bind_tools(tools)

def call_model(state: AgentState):
    response = model.invoke(state["messages"])
    return {"messages": [response]}

In [12]:
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [13]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# add edge from start to agent
workflow.add_edge(START, "agent")

# add conditional edge from agent to tools or end
workflow.add_conditional_edges("agent", should_continue, )

# add edge from tools to agent
workflow.add_edge("tools", "agent")

# compile the graph
app = workflow.compile()


In [14]:
from langchain_core.messages import HumanMessage

final_state = app.invoke({"messages": [HumanMessage(content="Who is the president of the USA?")]})
print(final_state["messages"][-1].content)

The current president of the USA is Donald John Trump, who assumed office on January 20, 2025. He is the 47th president of the United States and has been in office since January 20, 2025.


In [15]:
#pip install -U ddgs